# SQL query from table names - Continued

In [5]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [6]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [7]:
context = [ 
    {'role':'system', 'content':"""
     
 CREATE TABLE employees (
     ID_usr INT PRIMARY KEY,
     name VARCHAR(50)
);
INSERT INTO employees VALUES (1, 'Alice');
INSERT INTO employees VALUES (1, 'Bob');
INSERT INTO employees VALUES (1, 'Charlie');
     
 CREATE TABLE salary (
     ID_usr INT,
     year DATE,
     salary FLOAT,
     FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)     
);
INSERT INTO salary VALUES (1, '2023-01-01', 75000.00);
INSERT INTO salary VALUES (2, '2023-01-01', 85000.00);
INSERT INTO salary VALUES (3, '2023-01-01', 60000.00);

CREATE TABLE studies (
        ID INT PRIMARY KEY,
        ID_usr INT,
        educational_level INT,
        Institution VARCHAR(100),
        Years DATE,
        Speciality VARCHAR(100),
        FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);
INSERT INTO studies VALUES (1, 1, 5, 'Cambridge University', '2015-01-01', 'MBA');
INSERT INTO studies VALUES (2, 2, 6, 'MIT', '2018-01-01', 'BE');
INSERT INTO studies VALUES (3, 3, 4, 'Harvard University', '2016-01-01', 'MS IT');
    
"""} ]

In [8]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
                 
-- Question 1: What is the name of the employee with the highest salary?
    -- Expected SQL:
    SELECT employees.name
    FROM employees
    JOIN salary ON employees.ID_usr = salary.ID_usr
    ORDER BY salary.salary DESC
    LIMIT 1;

-- Question 2: What is the total salary for all employees in 2021?
    -- Expected SQL:
    SELECT SUM(salary.salary) AS total_salary
    FROM salary
    WHERE year = '2021-01-01';

-- Question 3: List all employees who have studied at Harvard University.
    -- Expected SQL:
    SELECT employees.name, studies.Institution
    FROM employees
    JOIN studies ON employees.ID_usr = studies.ID_usr
    WHERE studies.Institution = 'Harvard University';

-- Question 4: How many employees earned a salary of more than 70,000 in 2021?
    -- Expected SQL:
    SELECT COUNT(*) AS num_employees
    FROM salary
    WHERE salary.salary > 70000 AND year = '2021-01-01';
"""
})

In [10]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)



## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [13]:
#new
context_user = context.copy()
print(return_CCRMSQL("""What is the name of the employee with lowest salary?""", context_user))

```sql
SELECT employees.name
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr
ORDER BY salary.salary
LIMIT 1;
```


In [14]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("What is the name of the employee with lowest salary?", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary
LIMIT 1;
```

This SQL query selects the name of the employee with the lowest salary by joining the "employees" table with the "salary" table on the employee ID. It then orders the result by salary in ascending order and limits the output to the first row, which corresponds to the employee with the lowest salary.


In [15]:
#new
print(return_CCRMSQL("what is the average salary of employees from MIT?", context_user))

To find the average salary of employees who studied at MIT, you can use the following SQL query:

```sql
SELECT AVG(salary.salary) AS average_salary
FROM salary
JOIN studies ON salary.ID_usr = studies.ID_usr
WHERE studies.Institution = 'MIT';
```


In [16]:
#old 
print(return_CCRMSQL("what is the average salary of employees from MIT?", old_context_user))

This is your SQL:
```sql
SELECT AVG(s.salary) AS average_salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
JOIN studies st ON e.ID_usr = st.ID_usr
WHERE st.Institution = 'MIT';
```

This SQL query retrieves the average salary of employees from MIT by joining the employees, salary, and studies tables on the employee ID. It then filters the results to only include employees who studied at MIT and calculates the average salary of those employees.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [17]:
#new
print(return_CCRMSQL("what is the highest bonus in 2021?", context_user))

To answer the question about the highest bonus in 2021, we need to clarify whether the "bonus" information is stored in any of the existing tables. If the bonus information is not currently stored in the database schema provided, we would need to modify the schema to include a table that contains bonus information.

If we assume that the bonus information is stored in a separate table named "bonus" with columns `ID_usr`, `year`, and `bonus_amount`, we can provide the SQL query to find the highest bonus in 2021:

```sql
-- Question: What is the highest bonus in 2021?
SELECT MAX(bonus_amount) AS highest_bonus
FROM bonus
WHERE year = '2021-01-01';
```

Please let me know if you would like to add a "bonus" table to the schema, or if you have any other questions or requirements.


In [18]:
#old 
print(return_CCRMSQL("what is the highest bonus in 2021?", old_context_user))

This is your SQL:
```sql
SELECT MAX(salary) AS highest_bonus
FROM salary
WHERE year = '2021';
```

This SQL query selects the maximum salary value from the "salary" table where the year is 2021, giving you the highest bonus in 2021.


In [ ]:
# the new context provides additional information, when seeking information from a table, which was not defined